# ДОМАШНЯЯ РАБОТА - " РЕКОМЕНДАЦИИ НА ОСНОВЕ СОДЕРЖАНИЯ"

ДАНИЛЬЧЕНКО ВАДИМ

------------------------------------------------------------------------------------------------------------------------------

Задание:

1. Использовать dataset MovieLens
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:


- TF-IDF на тегах и жанрах
- Средние оценки (+ median, variance, etc.) пользователя и фильма


3. Оценить RMSE на тестовой выборке

------------------------------------------------------------------------------------------------------------------------------

подгрузим даные по фильмам и рейтингам

In [3]:
import pandas as pd
movies=pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings=pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


объединим в один

In [5]:
movie_ratings=movies.merge(ratings, on='movieId', how='left')
movie_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,3.0,8.551901e+08


подтянем тэги и объединим в один датасет

In [6]:
tags = pd.read_csv('tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
data2analyse=movie_ratings.merge(tags, on='movieId', how='left')
data2analyse.head()

,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,851866703.0,336.0,pixar,1.139046e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,851866703.0,474.0,pixar,1.137207e+09
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,851866703.0,567.0,fun,1.525286e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,938629179.0,336.0,pixar,1.139046e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,938629179.0,474.0,pixar,1.137207e+09


# обучим простую модель линейной регрессии на жанрах и тэгах, таргетом будет средняя оценка

In [24]:
# 3. tf-idf
from tqdm import tqdm_notebook

# составим список описания фильма, состоящий из тэгов и жанров
movies_descr=[]

# жанры
for i in tqdm_notebook(range(movies.shape[0])):
    movies_descr.append(movies.iloc[i,-1].split('|'))

# тэги
for i in tqdm_notebook(range(movies.shape[0])):
    movies_descr[i]=movies_descr[i]+list(set(tags[tags['movieId']==movies.iloc[i,0]].tag.tolist()))

movies_descr[:10]

[['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy', 'fun', 'pixar'],
 ['Adventure',
  'Children',
  'Fantasy',
  'magic board game',
  'game',
  'fantasy',
  'Robin Williams'],
 ['Comedy', 'Romance', 'old', 'moldy'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'remake', 'pregnancy'],
 ['Action', 'Crime', 'Thriller'],
 ['Comedy', 'Romance', 'remake'],
 ['Adventure', 'Children'],
 ['Action'],
 ['Action', 'Adventure', 'Thriller']]

In [56]:
# список средних оценок на фильм
rating_list=[]

for i in tqdm_notebook(range(movies.shape[0])):
    rating_list.append(ratings[ratings['movieId']==movies.iloc[i,0]].rating.mean())

In [37]:
# преобразуем в строку
movies_descr_str=[]
movies_descr_str.append([(' ').join(i) for i in movies_descr])
movies_descr_str[0][:5]

['Adventure Animation Children Comedy Fantasy fun pixar',
 'Adventure Children Fantasy magic board game game fantasy Robin Williams',
 'Comedy Romance old moldy',
 'Comedy Drama Romance',
 'Comedy remake pregnancy']

In [62]:
len(movies_descr_str[0])

9125

In [64]:
data_joined=pd.DataFrame(columns={'title', 'description', 'avg_rating'}, index=range(len(movies_descr_str[0])))
data_joined['title']=movies.title
data_joined['description']=movies_descr_str[0]
data_joined['avg_rating']=rating_list
data_joined.head()

,avg_rating,description,title
0,3.872470,Adventure Animation Children Comedy Fantasy fu...,Toy Story (1995)
1,3.401869,Adventure Children Fantasy magic board game ga...,Jumanji (1995)
2,3.161017,Comedy Romance old moldy,Grumpier Old Men (1995)
3,2.384615,Comedy Drama Romance,Waiting to Exhale (1995)
4,3.267857,Comedy remake pregnancy,Father of the Bride Part II (1995)


In [67]:
# сгруппируем на всякий случай по description и возьмем среднее
# ps: знаю, что надо бы взвесить по количетсву оценок, но времени уже не осталось
data_joined2=data_joined.groupby('description').avg_rating.mean().reset_index()
data_joined2.head()

,description,avg_rating
0,(no genres listed),3.720588
1,Action,2.707357
2,Action Adventure,3.348586
3,Action Adventure Animation,3.854167
4,Action Adventure Animation Children,3.525641


In [68]:
# 3. tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector=TfidfVectorizer()

X=tfidf_vector.fit_transform(data_joined2.description)

k=0
print('словарик tf-idf:')
print('--------------------')
for key, val in tfidf_vector.vocabulary_.items():
    print({key:val})
    k+=1
    if k>=10:
        break

словарик tf-idf:
--------------------
{'no': 1052}
{'genres': 609}
{'listed': 886}
{'action': 21}
{'adventure': 34}
{'animation': 74}
{'children': 274}
{'comedy': 316}
{'disney': 430}
{'superhero': 1453}


In [113]:
from sklearn.cross_validation import train_test_split
data_train, data_test, y_train, y_test=train_test_split(X.toarray(), data_joined2.avg_rating.fillna(0), test_size=0.3)

In [120]:
# обучим модель линейной регрессии
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(data_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [121]:
# предскажем оценки по тренировочной и тестовой выборке
y_train_pred=model.predict(data_train)
y_test_pred=model.predict(data_test)

# посчитаем RMSE
from sklearn.metrics import mean_squared_error

print(f'RMSE train: {mean_squared_error(y_train, y_train_pred)}')
print(f'RMSE test: {mean_squared_error(y_test, y_test_pred)}')

RMSE train: 0.2495233019219977
RMSE test: 6.6063947742237035e+25


очень большая ошибка на тесте, я бы такую модель никому не посоветовал

# а теперь лучше построим по tf-idf топ 10 рекомендаций по ближайшим соседям

вектор построим новый

In [206]:
tfidf_vector2=TfidfVectorizer()
X_train_tfidf=tfidf_vector2.fit_transform(data_joined.description)

In [207]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='cosine') 
neigh.fit(X_train_tfidf)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=-1, n_neighbors=10, p=2, radius=1.0)

In [212]:
# подберем ближаших соседей по описанию бойцовского клуба
test_movie=movies[movies['title'].str.contains('10th Kingdom')].movieId
test_movie_list=[]

# подтянем жанры
test_movie_list.append(str(list(movies[movies.movieId==list(test_movie)].genres)[0]).split('|'))
# test_movie_list=test_movie_list+[i for i in tags[tags.movieId==list(test_movie)].tag.tolist()]
test_movie_list[0]=test_movie_list[0]+([i for i in list(set(tags[tags.movieId==list(test_movie)].tag.tolist()))])
test_movie_list[0]

['Adventure', 'Comedy', 'Fantasy']

In [213]:
test_tfidf=tfidf_vector2.transform(test_movie_list[0])
res = neigh.kneighbors(test_tfidf, return_distance=True)
res

(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.09776268, 0.09776268, 0.09776268,
         0.09776268, 0.09776268, 0.09776268, 0.09776268, 0.09776268]]),
 array([[5494, 5723, 6993, 4349, 4093, 2579, 2545, 3867, 4090, 2034],
        [2945, 2944, 6427, 8483, 8488, 4303, 4307, 4286,  959, 8505],
        [8067, 8066, 2141, 8418, 7655, 5431,  177, 5855, 7278, 2099]],
       dtype=int64))

In [214]:
movies.iloc[res[1][0]]

,movieId,title,genres
5494,8654,Prince Valiant (1954),Adventure
5723,26013,Rodan (Sora no daikaijû Radon) (1956),Adventure
6993,60343,Wee Willie Winkie (1937),Adventure
4349,5864,"Tarzan, the Ape Man (1981)",Adventure
4093,5361,White Fang (1991),Adventure
2579,3207,"Snows of Kilimanjaro, The (1952)",Adventure
2545,3166,Brenda Starr (1989),Adventure
3867,4988,White Water Summer (1987),Adventure
4090,5357,Iron Will (1994),Adventure
2034,2537,Beyond the Poseidon Adventure (1979),Adventure


мы получили ТОП 10 фильмов, наиболее близких к выбранному

ps: правда немного цепляет глаз, что в жанрах только Adventure, но уже нет времени разбираться